In [1]:
import os
import time
import urllib
import pandas as pd
from PIL import Image
from pathlib import Path

In [2]:
new_df=pd.DataFrame(columns=["image_path", "group_id", "entity_name", "entity_value"])

In [3]:
def create_placeholder_image(image_save_path, group_id, entity_name, entity_value):

    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
        
        new_row={
            "image_path":image_save_path,
            "group_id":group_id,
            "entity_name":entity_name,
            "entity_value":entity_value
        }

        new_df.loc[len(new_df)] = new_row
        return
        
    except Exception as e:
        print("Exception is:", e)
        return

In [4]:
def download_image(row, save_folder, retries=3, delay=3):
    
    image_link=row['image_link']
    group_id=row['group_id']
    entity_name=row['entity_name']
    entity_value=row['entity_value']

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            new_row={
                "image_path":image_save_path,
                "group_id":group_id,
                "entity_name":entity_name,
                "entity_value":entity_value
            }
            new_df.loc[len(new_df)] = new_row
            return
        
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path, group_id, entity_name, entity_value)

In [5]:
df=pd.read_csv('./dataset/train.csv').head(1000)
for index, row in df.iterrows():
    download_image(row, "./images/")

In [6]:
new_df.to_csv("img_path_dataset.csv", index=False)